## Example live anaylsis of audio from your mic

In [ ]:
%matplotlib widget
from ipywebrtc import AudioRecorder, CameraStream
from matplotlib.animation import FuncAnimation
from threading import Thread # More threads
import io 
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
import scipy.fft as fft
import time 
import sys
import av

### Record 
This uses your microphone and only stores it to memory.
You have to give permission in the browser popup to take up the sound

In [ ]:
fs = 48000 # The sampling rate

camera = CameraStream(constraints={'audio': True, 'video':False, 'sampleRate':fs}) 
# We are not recoring a video, only sound, and it is not stored.
camera.playing=False
recorder = AudioRecorder(stream=camera, format='webm', muted=True)


### Methods for recording and processing the recording
The way we are doing the recording means that there will be some lag, and we will be processing on previous data. 

In [ ]:
# This is to start with some data
recorder.recording = True # Start the recording
time.sleep(1)
recorder.recording = False # Stop the recording

def record(first = True,interval = 100):
    # Method for taking a new recording
    recorder.recording = True # Start the recording
    time.sleep(interval/1000)
    recorder.recording = False # Stop the recording
    if (len(recorder.audio.value) == 0) and first:
        # Tryiung one more time, as there is some lag
        return record(first=False)
    elif (len(recorder.audio.value) == 0) and (not(first)):
        raise RuntimeError("Not able to record")
    else: 
        # Convert the audio data  in the memmory into a NumPy array
        container = av.open(io.BytesIO(recorder.audio.value))
        for idx, frame in enumerate(container.decode()):
            if idx==0:
                audio = frame.to_ndarray()
            else:
                audio=np.concatenate((audio, frame.to_ndarray()),axis=1)
        return np.squeeze(audio[0,:]), fs

def process(recording, N, fs):
    # Process the recording 
    X = fft.fftshift(fft.fft(recording, n=N)) # Taking the FFT and shifting it for display
    f = fft.fftshift(fft.fftfreq(N, d = 1/fs)) # Calculating the returned frequencies and shifting it
    return 20*np.log10(np.abs(X)), f




### Plotting the data

In [ ]:
class LiveGraph:
    def __init__(self):
        # Getting first audio
        self.N = 4096
        self.audio, self.fs = record()
        print(self.audio.shape)
        self.X_mag, self.f = process(self.audio, self.N, self.fs)
        print(self.fs)

        # Making the figure
        self.fig, self.ax = plt.subplots(2, 1, figsize=[8, 8])

        # Making the sample index
        self.n = np.arange(len(self.audio))

        # Plotting the waveform
        self.line0, = self.ax[0].plot(self.n, self.audio) #, 'o', markerfacecolor='orange')
        self.line1, = self.ax[1].plot(self.f, self.X_mag) #, 'o', markerfacecolor='orange')

        # Setting the axes 
        self.ax[0].set_xlabel('Time ($n$)')
        self.ax[0].set_ylabel('Amplitude')
        self.ax[0].set_xlim([0,self.n[-1]])
        self.ax[0].set_ylim([-1,1])
        self.ax[0].grid()

        self.ax[1].set_xlabel('Frequency (Hz)')
        self.ax[1].set_ylabel('Magnitude (dB)')
        self.ax[1].set_xlim([0, self.N//2-1])
        self.ax[1].set_ylim([-40,60])
        self.ax[1].grid()
        
        self.interval = 200
        self.animation = FuncAnimation(self.fig, self.update, frames = np.arange(0,400), interval=self.interval)
        self.th = Thread(target=self.thread_f, daemon=True)
        self._stop = False
        self.th.start()
        


    def update(self,frame):
        # Updating the graph with any new data
        self.line0.set_data(self.n, self.audio)
        self.line1.set_data(self.f, self.X_mag)
        
        self.ax[0].set_xlim([0,np.max(self.n)])
        self.ax[0].set_ylim([-1,1])
        
        
        #self.ax[1].set_ylim([0, np.max(self.X_mag)*1.1])
        
        return self.line0


    def show(self):
        plt.show()

    def stop(self):
        self._stop =True
        time.sleep(0.5) 
        self.th._stop()
        self.animation.pause()

    def thread_f(self):
        n = 0
        while n<400:
            if self._stop:
                break    
            self.audio, self.fs = record(self.interval)
            self.n = np.arange(len(self.audio))
            self.X_mag, self.f = process(self.audio, self.N, self.fs)
            n += 1
            time.sleep(self.interval/1000)  
        
          


### The lines below will start the analysis.

In [ ]:
#recorder.recording = True
try: 
    g = LiveGraph()
    g.show()
except RuntimeError:
    try: 
        time.sleep(0.4)
        g = LiveGraph()
        g.show()
    except RuntimeError:
        print("Try again  ")

### Run the lines below to stop everything

In [ ]:
g.stop()
g.animation.pause